In [398]:
import importlib
import utils
import transformers

In [399]:
from utils import *

In [388]:
importlib.reload(utils)
importlib.reload(transformers)

<module 'transformers' from 'd:\\Documents\\machinelearning\\cgm-paper\\src\\transformers.py'>

In [389]:
print("Current working directory: ", os.getcwd())

Current working directory:  d:\Documents\machinelearning\cgm-paper\src


In [390]:
raw_dir = "../data/raw"
tuning_dir = "../data/tuning"

In [391]:
assert os.path.exists(raw_dir), "Raw data directory not found"
assert os.path.exists(tuning_dir), "Tuning data directory not found"

In [392]:
dataset = load_data(raw_dir)

Found 22 files in ../data/raw
train (139415, 47)
test (32912, 47)


In [393]:
val_dataset = load_data(tuning_dir)

Found 2 files in ../data/tuning
train (13630, 43)
test (3002, 43)


In [394]:
for key in dataset.keys():
    val_dataset[key] = reduce_classes(val_dataset[key])
    dataset[key] = reduce_classes(dataset[key])

In [395]:
dataset.get("train")['Time'].isnull().sum()

0

In [396]:
train_data = get_train_dataset(dataset['train'])
test_data = get_any_dataset(dataset['test'], dataset['train'])

The tuning data will be merged with the training data and be fitted into the pipeline to be used for the validation data

In [397]:
tuning_data = get_tuning_dataset(val_dataset['train'], dataset['train'])
val_data = get_any_dataset(val_dataset['test'], tuning_data)